# Fake ETL Pipeline: Pandas Skills Reinforcement

**Project Goal:**  
Simulate an end-to-end data engineering workflow using synthetic CSV data to practice and showcase core Pandas techniques:
- **Data cleaning:** strip whitespace, handle missing values, coerce types  
- **Data transformation & reshaping:** `pivot_table` for summary, `melt` for tidy formats  
- **Time series handling:** parse and resample date/time fields  
- **Type validation:** numeric conversion and date parsing  

**Datasets:**  
- **`raw_data/users.csv`** – synthetic user profiles (`user_id`, `name`, `signup_date`)  
- **`raw_data/transactions.csv`** – synthetic transaction logs (`txn_id`, `user_id`, `amount`, `timestamp`)  

Throughout this notebook I will:
1. Load & inspect raw data  
2. Clean each table (strip, dropna, parse dates, to_numeric)  
3. Merge into a master table  
4. Produce summary tables (monthly spend per user, weekly revenue)  
5. Export cleaned and reshaped outputs under `cleaned/` and `output_summary/`

In [1]:
# Step 1: Import pandas load csv file & inspect 
import pandas as pd
tx_df = pd.read_csv("raw_data/transactions.csv", parse_dates=["timestamp"])
users_df = pd.read_csv("raw_data/users.csv", parse_dates=["signup_date"])

In [2]:
# inspect top 5 rows of data
tx_df.head()

,txn_id,user_id,amount,timestamp
0,101,1,120.5,2025-06-01 10:00:00
1,102,2,85.0,2025-06-05 12:30:00
2,103,1,75.2,2025-06-07 09:15:00
3,104,3,150.0,2025-06-08 14:20:00
4,105,4,200.0,2025-06-10 18:45:00


In [3]:
users_df.head()

,user_id,name,signup_date
0,1,Alice,2025-01-15
1,2,Bob,2025-02-20
2,3,Eve,2025-03-12
3,4,Mallory,2025-04-18
4,5,Ken,2025-04-20


In [4]:
# check any non-null value and datatype, ensure parse_date sucessfully
tx_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   txn_id     5 non-null      int64         
 1   user_id    5 non-null      int64         
 2   amount     5 non-null      float64       
 3   timestamp  5 non-null      datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 288.0 bytes


In [5]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   user_id      7 non-null      int64         
 1   name         6 non-null      object        
 2   signup_date  7 non-null      datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 296.0+ bytes


In [6]:
# Step 2: Clean Users
users_df["name"] = users_df["name"].str.strip() # strip whitespace from name column
users_df = users_df.drop_duplicates() # remove any duplicate record
users_df = users_df.dropna(subset=['user_id','name']) # remove the record where user_id and name is missing value
users_df

,user_id,name,signup_date
0,1,Alice,2025-01-15
1,2,Bob,2025-02-20
2,3,Eve,2025-03-12
3,4,Mallory,2025-04-18
4,5,Ken,2025-04-20


In [7]:
# Create "cleaned" and "output_summary" directories 
import os 
os.makedirs("cleaned", exist_ok=True)
os.makedirs("output_summary", exist_ok=True)

In [8]:
# Export cleaned users dataframe to csv into "cleaned" directory
users_df.to_csv("cleaned/users_clean.csv", index=False)

In [9]:
# Step 3: Clean Transactions
tx_df['amount'] = pd.to_numeric(tx_df['amount'], errors='coerce') # convert column "amount" to numeric data type, if the data not valid wil change to NaN
tx_df = tx_df.drop_duplicates()
tx_df = tx_df.dropna(subset=['txn_id', 'user_id','amount'])
tx_df

,txn_id,user_id,amount,timestamp
0,101,1,120.5,2025-06-01 10:00:00
1,102,2,85.0,2025-06-05 12:30:00
2,103,1,75.2,2025-06-07 09:15:00
3,104,3,150.0,2025-06-08 14:20:00
4,105,4,200.0,2025-06-10 18:45:00


In [10]:
# export cleaned transactions dataframe to csv
tx_df.to_csv('cleaned/transactions_clean.csv', index=False)

In [11]:
# Step 4: Merge cleaned transactions & users df -> master_df 
df_clean_txn = pd.read_csv("cleaned/transactions_clean.csv", parse_dates=["timestamp"])
df_clean_user = pd.read_csv("cleaned/users_clean.csv", parse_dates=["signup_date"])
master_df = df_clean_user.merge(df_clean_txn, on="user_id", how="inner" ) # using column user_id to join table (inner join)
master_df.head()

,user_id,name,signup_date,txn_id,amount,timestamp
0,1,Alice,2025-01-15,101,120.5,2025-06-01 10:00:00
1,1,Alice,2025-01-15,103,75.2,2025-06-07 09:15:00
2,2,Bob,2025-02-20,102,85.0,2025-06-05 12:30:00
3,3,Eve,2025-03-12,104,150.0,2025-06-08 14:20:00
4,4,Mallory,2025-04-18,105,200.0,2025-06-10 18:45:00


In [12]:
# export master dataframe to csv
master_df.to_csv("cleaned/master.csv", index=False)

In [13]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   user_id      5 non-null      int64         
 1   name         5 non-null      object        
 2   signup_date  5 non-null      datetime64[ns]
 3   txn_id       5 non-null      int64         
 4   amount       5 non-null      float64       
 5   timestamp    5 non-null      datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(2), object(1)
memory usage: 368.0+ bytes


In [14]:
# Step 5: Summary the data
# create users monthly spend pivot table
master_df['month'] = master_df['timestamp'].dt.to_period('M') #Create new column "month" period
usr_monthly_spend_table = master_df.pivot_table(values="amount", index=["user_id","name"], columns=["month"], aggfunc="sum", fill_value=0) 

In [15]:
usr_monthly_spend_table.reset_index()
usr_monthly_spend_table

,month,2025-06
user_id,name,
1,Alice,195.7
2,Bob,85.0
3,Eve,150.0
4,Mallory,200.0


In [16]:
# from wide form change back to long form -> user_id, name, month, total_spend
monthly_longform = usr_monthly_spend_table.reset_index().melt(id_vars=["user_id","name"], var_name="month",value_name="total_spend" )

In [17]:
monthly_longform

,user_id,name,month,total_spend
0,1,Alice,2025-06,195.7
1,2,Bob,2025-06,85.0
2,3,Eve,2025-06,150.0
3,4,Mallory,2025-06,200.0


In [18]:
# set column timestamp to index, using resample method to calculate total revenue per week.
weekly_revenue = (
    master_df
        .set_index("timestamp")["amount"]
        .resample('W').sum()
        .reset_index(name="total_revenue")
)

In [19]:
weekly_revenue

,timestamp,total_revenue
0,2025-06-01,120.5
1,2025-06-08,310.2
2,2025-06-15,200.0


In [20]:
# Export usr_monthly_spend_table(wide form), monthly_longform(long form), weekly_revenue to csv files

usr_monthly_spend_table.to_csv("output_summary/user_monthly_spend.csv")
monthly_longform.to_csv("output_summary/user_monthly_spend_long.csv",index=False)
weekly_revenue.to_csv("output_summary/weekly_revenue.csv", index=False)                        

In [21]:
import json
import pandas as pd
#1 Load raw json
with open("raw_data/data.json", "r") as f:
    raw = json.load(f)

#2 Convert each list of dicts into a DataFrame
users_json_df = pd.DataFrame(raw["users"])
txn_json_df = pd.DataFrame(raw["transactions"])

#3 Clean, parse dates and types
users_json_df["name"] = users_json_df["name"].str.strip()
users_json_df = users_json_df.replace("", pd.NA).dropna(subset=["user_id", "name"])
users_json_df = users_json_df.drop_duplicates(subset=["user_id", "name"])
users_json_df["signup_date"] = pd.to_datetime(users_json_df["signup_date"])
txn_json_df["timestamp"] = pd.to_datetime(txn_json_df["timestamp"])
txn_json_df["amount"] = pd.to_numeric(txn_json_df["amount"], errors='coerce')
txn_json_df = txn_json_df.dropna(subset=["txn_id", "user_id", "amount"])

#4 Inspect
print(users_json_df.head(), users_json_df.dtypes, sep="\n")
print(txn_json_df.head(), txn_json_df.dtypes, sep="\n")

   user_id     name signup_date
0        1    Alice  2025-01-15
1        2      Bob  2025-02-20
2        3      Eve  2025-03-12
3        4  Mallory  2025-04-18
4        5      Ken  2025-04-20
user_id                 int64
name                   object
signup_date    datetime64[ns]
dtype: object
   txn_id  user_id  amount           timestamp
0     101        1   120.5 2025-06-01 10:00:00
1     102        2    85.0 2025-06-05 12:30:00
2     103        1    75.2 2025-06-07 09:15:00
3     104        3   150.0 2025-06-08 14:20:00
4     105        4   200.0 2025-06-10 18:45:00
txn_id                int64
user_id               int64
amount              float64
timestamp    datetime64[ns]
dtype: object


In [22]:
# Merge users & txn JSON-based DataFrame
master_json_df = users_json_df.merge(txn_json_df, on="user_id", how="inner")
print(master_json_df.head(), master_json_df.dtypes, sep="\n")

   user_id     name signup_date  txn_id  amount           timestamp
0        1    Alice  2025-01-15     101   120.5 2025-06-01 10:00:00
1        1    Alice  2025-01-15     103    75.2 2025-06-07 09:15:00
2        2      Bob  2025-02-20     102    85.0 2025-06-05 12:30:00
3        3      Eve  2025-03-12     104   150.0 2025-06-08 14:20:00
4        4  Mallory  2025-04-18     105   200.0 2025-06-10 18:45:00
user_id                 int64
name                   object
signup_date    datetime64[ns]
txn_id                  int64
amount                float64
timestamp      datetime64[ns]
dtype: object


In [23]:
# Export master JSON-based DataFrame to JSON
master_json_df.to_json("cleaned/master.json", orient="records", date_format="iso", indent=4)